In [4]:
from evaluator import SyntacticCalculator, MorphologicalCalculator, InventoryCalculator, PhonologicalCalculator, FeaturalCalculator, GenericCalculator, LangRankEvaluator, IslandCalculator
script = GenericCalculator('data/URIEL_Scriptural.csv')
islands = IslandCalculator('data/URIELPlus_Union_SoftImpute.csv')
geo = GenericCalculator('data/URIEL_Geography.csv')
gen = GenericCalculator('data/URIEL_Phylogeny.csv')
typ_file = 'data/URIELPlus_Union.csv' # path to the typological dataset CSV file
syn = SyntacticCalculator(typ_file)
morph = MorphologicalCalculator(typ_file)
inv = InventoryCalculator(typ_file)
phon = PhonologicalCalculator(typ_file)
feat = FeaturalCalculator(typ_file)
evaluator = LangRankEvaluator(
    calculators={
    'syntactic': syn,
    'morphological': morph,
    'inventory': inv,
    'phonological': phon,
    'featural': feat,
    'scriptural': script,
    'islands': islands,
    'geographic': geo,
    'genetic': gen,
    },
    iso_map_file='data/code_mapping.csv', # path to the ISO to Glottocode mapping file
)

In [ ]:
baseline_ndcg = {
    'mt': [],
    'dep': [],
    'el': [],
    'pos': [],
    'taxi1500': [],
    'xnli': []
}

In [14]:
is_baseline = True
features = ['syntactic', 'phonological', 'inventory', 'featural', 'geographic', 'genetic']
# features += ['morphological']
# features += ['scriptural']
# features += ['islands']
task_col_name = 'task_lang'
transfer_col_name = 'transfer_lang'

tasks = {
    'mt': ('BLEU', True),
    'dep': ('accuracy', True),
    'el': ('accuracy', True),
    'pos': ('accuracy', True),
    # 'taxi1500': ('f1_score', False),
    'xnli': ('accuracy', False),
}
for task in tasks:
    # replace/add distances in the `distance_types` columns of the dataset CSV using the corresponding calculators passed into the evaluator.
    evaluator.replace_distances(
        dataset_path=f'data/{task}.csv', # path to the task dataset (the one containing columns for task lang, transfer lang, performance)
        distance_types=features, # list of distance types to replace in the dataset. these should match the keys of the dict passed into the evaluator.
        task_col_name=task_col_name, # name of the task language column in your dataset
        transfer_col_name=transfer_col_name, # name of the transfer language column in your dataset
        iso_conversion=tasks[task][1] # indicate whether to convert lang codes in your dataset from ISO to Glottocode using the file in self.iso_map_file
    )
    # run LangRank and evaluate task performance
    score = evaluator.evaluate(
        dataset_path=f'data/{task}_updated.csv',
        features=features, # list of columns in the dataset CSV to use for evaluation
        performance_col_name=tasks[task][0], # name of the column in the dataset CSV containing task performance scores
        task_col_name=task_col_name,
        transfer_col_name=transfer_col_name,
        baseline_ndcg_scores=baseline_ndcg[task], # list of baseline NDCG scores for the task
    )
    if is_baseline:
        baseline_ndcg[task].append(score[1])
        print(f'Task: {task} NDCG: {score[0]}')
    else:
        print(f'Task: {task} NDCG: {score[0]} (p-value: {score[2]})')

KeyboardInterrupt: 